With access to the current open map through `aprx.activeView` we can do all kinds of fun things like:
* access the underlying map object
* change the camera

In [15]:
import time
import ipywidgets as widgets

In [16]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
mv = aprx.activeView # mapview
print(type(mv))
try:
    m = mv.map # map
except(AttributeError):
    print("No map found. Open a map alongside this notebook.")


<class 'arcpy._mp.MapView'>


In [17]:
# function for moving the camera to chosen number of features
# in the chosen layer, with optional labels!

def tour(my_layer,n,labels):
    my_layer = m.listLayers(my_layer)[0]
    desc = arcpy.Describe(my_layer)
    oid_field = desc.OIDFieldName
    if labels:
        my_layer.showLabels = True
    for i in range(n):
        arcpy.SelectLayerByAttribute_management(my_layer.name,
                                                "NEW_SELECTION", 
                                                oid_field+"="+str(i+1))
        # getLayerExtent honors selection by default
        mv.camera.setExtent(mv.getLayerExtent(my_layer))
        if desc.shapeType == "Point":
            mv.camera.scale=300000
        time.sleep(1)

    # remove selection
    my_layer.setSelectionSet()
    if labels:    # turn off labels
        my_layer.showLabels = False
    # zoom to full extent
    mv.camera.setExtent(mv.getLayerExtent(my_layer))
    
# tour("BusStops",5,False)

ipywidgets can be interactive and linked to each other.

In [18]:
mv = aprx.activeView # mapview



try:
    m = mv.map # map
    print(f"{m.name} Tour")
except(AttributeError):
    sys.exit("No map found. Open a map alongside this notebook.")
    
valid_layers=len([l.name for l in m.listLayers("*") if arcpy.Describe(l).dataElement])

if valid_layers<1:
    sys.exit("No valid layers, feature layer required for tour.")

layer_menu = widgets.Dropdown(options=[l.name for l in m.listLayers("*") if arcpy.Describe(l).dataElement]
                             , )#value=None)

slider = widgets.IntSlider(value=5,min=1,max=10)

labels = widgets.ToggleButton(
    value=True,
    description='Labels',
    disabled=False)

# create button to call the function
button = widgets.Button(icon='play')
button.on_click(lambda x: tour(layer_menu.value,slider.value,labels.value))

# link slider with dropdown so it shows the correct count
widgets.dlink((layer_menu,"value"),(slider,"max"),
                      lambda x:int(arcpy.GetCount_management(layer_menu.value)[0]))

# display all widgets
widgets.VBox([layer_menu,slider,labels,button])



California Coastline Tour
